#### Business Analytics FHDW 2025
# Time Series
## Regressionsbasierte Vorhersagen

Wir nutzen wieder *Amtrak.csv* mit seinen Fahrgastzahlen. Die *ridership*-Zeitreihe leiten wir uns wie im Beispiel davor daraus ab und transformieren auch wieder die Monatsdaten in einen Zeitindex. Die etwas aufwändigere, wiederkehrende grafische Darstellung packen wir zu Teilen in Funktionen `graphLayout` und `singleGraphLayout`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import statsmodels.formula.api as smf
from statsmodels.tsa import tsatools
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics import tsaplots

def regressionSummary(y_true, y_predicted):
    y_true = np.asarray(y_true)
    y_predicted = np.asarray(y_predicted)
    y_residuals = y_true - y_predicted
    metrics = [
        ('Summe Abweichungen', sum(y_residuals)),
        ('Summe absolute Abweichungen', sum(abs(y_residuals))),
        ('Mittlerer Fehler', sum(y_residuals) / len(y_residuals)),
        ('Mittlerer absoluter Fehler', sum(abs(y_residuals)) / len(y_residuals)),
        ('Wurzel des durchschnittlichen Fehlerquadrats', np.sqrt(mean_squared_error(y_true, y_predicted)))
    ]
    if all(yt != 0 for yt in y_true):
        metrics.extend([
            ('Mittlerer prozentualer Fehler', 100 * sum(y_residuals / y_true) / len(y_residuals)),
            ('Mittlerer absoluter prozentualer Fehler', 100 * sum(abs(y_residuals / y_true) / len(y_residuals))),
        ])
    maxlength = max(len(m[0]) for m in metrics)
    fmt1 = f'{{:>{maxlength}}} : {{:.4f}}'    
    for metric, value in metrics:
        print(fmt1.format(metric, value))

def singleGraphLayout(ax, ylim, train_df, valid_df):
    ax.set_xlim('1990', '2004-6')
    ax.set_ylim(*ylim)
    ax.set_xlabel('Zeit')
    one_month = pd.Timedelta('31 days')
    
    xtrain = (min(train_df.index), max(train_df.index)-one_month)
    xvalid = (min(valid_df.index)+one_month, max(valid_df.index)-one_month)
    xtv = xtrain[1]+0.5*(xvalid[0]-xtrain[1])
    
    ypos = 0.9*ylim[1]+0.1*ylim[0]
    ax.add_line(plt.Line2D(xtrain, (ypos, ypos), color='black', linewidth=0.5))
    ax.add_line(plt.Line2D(xvalid, (ypos, ypos), color='black', linewidth=0.5))
    ax.axvline(x=xtv, ymin=0, ymax=1, color='black', linewidth=0.5)
    
    ypos = 0.925*ylim[1]+0.075*ylim[0]
    ax.text('1995', ypos, 'Training')
    ax.text('2002-3', ypos, 'Validation')
    
def graphLayout(axes, train_df, valid_df):
    singleGraphLayout(axes[0], [1300, 2550], train_df, valid_df)
    singleGraphLayout(axes[1], [-550, 550], train_df, valid_df)
    train_df.plot(y='Ridership', ax=axes[0], color='C0', linewidth=0.75)
    valid_df.plot(y='Ridership', ax=axes[0], color='C0', linestyle='dashed', linewidth=0.75)
    axes[1].axhline(y=0, xmin=0, xmax=1, color='black', linewidth=0.5)
    axes[0].set_xlabel('')
    axes[0].set_ylabel('Fahrgäste in tausend')
    axes[1].set_ylabel('Vorhersagefehler')
    if axes[0].get_legend():
        axes[0].get_legend().remove()

amtrak_df = pd.read_csv('Daten/Amtrak.csv')
amtrak_df['Date'] = pd.to_datetime(amtrak_df.Month, format='%d/%m/%Y')
ridership_ts = pd.Series(amtrak_df.Ridership.values, index=amtrak_df.Date, name='Ridership')
ridership_ts

Auf die Zeitreihe passen wir ein einfaches lineares Trendmodell an. Dazu nutzen wir in diesem Fall *tsatools*, was uns Werkzeuge spezifisch zur Arbeit mit Zeitreihen bietet. `add_trend` erzeugt aus der Fahrgästezeitreihe ein DataFrame und erweitert es mit dem Parameter `trend='t'` um eine lineare Prädiktorvariable mit einem Wert für jeden Zeitschritt (auch die zeitbezogene Regression arbeitet mit den bekannten Formeln; mit der Kodierung `datetime64` von *Date* können diese nicht viel anfangen, daher die Vereinfachung/Übersetzung). 

In [ ]:
ridership_df = tsatools.add_trend(ridership_ts, trend='t')
ridership_df

Diesen Datensatz können wir nun als Eingabe für eine *ordinary least squares*-Regression `ols` nutzen. *statsmodels* erlaubt uns, dazu explizit eine Formel mit der Vorhersage `Ridership` in Abhängigkeit vom Prädiktor `trend` zu definieren. `Ridership ~ trend` entspricht dabei $Y_t = \beta_0 + \beta_1 t + \epsilon$. Mit `fit` nimmt das resultierende Modell dann die Anpassung vor. Das Ergebnis stellen wir dar.

In [ ]:
ridership_lin_reg = smf.ols(formula='Ridership ~ trend', data=ridership_df).fit()
ax = ridership_ts.plot()
ax.set_xlabel('Zeit')
ax.set_ylabel('Fahrgastzahlen in tausend')
ax.set_ylim(1300, 2300)
ridership_lin_reg.predict(ridership_df).plot(ax=ax)
plt.show()

Auf dieser Basis können wir auch ein Modell für Vorhersagen bauen. Dazu teilen wir den oben um den Prädiktor erweiterten Datensatz wie gewohnt auf. Das Ergebnis visualisieren wir und die Kennzahlen geben wir auch aus.

In [ ]:
n_valid = 36 
n_train = len(ridership_ts) - n_valid
train_df = ridership_df[:n_train]
valid_df = ridership_df[n_train:]

ridership_prediction_linear = smf.ols(formula='Ridership ~ trend', data=train_df).fit()
predicted_linear = ridership_prediction_linear.predict(valid_df)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 7.5))
ridership_prediction_linear.predict(train_df).plot(ax=axes[0], color='C1')
ridership_prediction_linear.predict(valid_df).plot(ax=axes[0], color='C1', linestyle='dashed')

residual = train_df.Ridership - ridership_prediction_linear.predict(train_df)
residual.plot(ax=axes[1], color='C1')
residual = valid_df.Ridership - ridership_prediction_linear.predict(valid_df)
residual.plot(ax=axes[1], color='C1', linestyle='dashed')
graphLayout(axes, train_df, valid_df)
plt.tight_layout()
plt.show()

Für das Ergebnis lassen wir uns eine Auswertung ausgeben:

In [ ]:
print(ridership_prediction_linear.summary())

Der *Intercept* ist der Wert der Konstante $\beta_0$, *trend* der Koeffizient $\beta_1$. An ihren ggf. akzeptablen statistischen Kennzahlen sehen wir hier, dass die Betrachtung der Form des zeitlichen Verlaufs erforderlich ist, nicht nur der reinen Zahlen, denn offensichtlich gibt es hier keinen linearen Zusammenhang. 

Also versuchen wir es als nächstes mit einem *exponentiellem Trend*. Praktisch nutzen wir hier die Umformung von $Y_t = ce^{\beta_1 t + \epsilon}$ zu $\log Y_t = \beta_0 + \beta_1 t + \epsilon$. Bei der Auswertung und Darstellung müssen wir dann aber beachten, die Trainingsdaten und Vorhersagen auf die ursprüngliche Skala abzubilden. Dazu wenden wir hier die Exponentialfunktion als $\lambda$ auf die entsprechenden Werte an. 

Im Vergleich stellt sich dann ein sehr ähnliches Ergebnis wie im linearen Modell dar:

In [ ]:
ridership_prediction_expo = smf.ols(formula='np.log(Ridership) ~ trend', data=train_df).fit()
predicted_expo = ridership_prediction_expo.predict(valid_df).apply(lambda row: np.exp(row))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 3.75))
train_df.plot(y='Ridership', ax=ax, color='C0', linewidth=0.75)
valid_df.plot(y='Ridership', ax=ax, color='C0', linestyle='dashed', linewidth=0.75)
singleGraphLayout(ax, [1300, 2600], train_df, valid_df)
ridership_prediction_linear.predict(train_df).plot(color='C1')
ridership_prediction_linear.predict(valid_df).plot(color='C1', linestyle='dashed')
ridership_prediction_expo.predict(train_df).apply(lambda row: np.exp(row)).plot(color='C2')
ridership_prediction_expo.predict(valid_df).apply(lambda row: np.exp(row)).plot(color='C2', linestyle='dashed')
ax.get_legend().remove()
plt.show()

Passen wir als nächstes einen *polynomialen Trend* an. Dazu fügen wir einen weiteren Prädiktor $t^2$ in die Formel ein. Dieser quadratische Zusammenhang ist hier nahe liegend, da der Verlauf der Daten "U-förmig" wirkt.

In [ ]:
ridership_prediction_poly = smf.ols(formula='Ridership ~ trend + np.square(trend)', data=train_df).fit()
predicted_poly = ridership_prediction_poly.predict(valid_df)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 7.5))
ridership_prediction_poly.predict(train_df).plot(ax=axes[0], color='C1')
ridership_prediction_poly.predict(valid_df).plot(ax=axes[0], color='C1', linestyle='dashed')

residual = train_df.Ridership - ridership_prediction_poly.predict(train_df)
residual.plot(ax=axes[1], color='C1')
residual = valid_df.Ridership - ridership_prediction_poly.predict(valid_df)
residual.plot(ax=axes[1], color='C1', linestyle='dashed')
graphLayout(axes, train_df, valid_df)
plt.show()

Das Ergebnis wirkt nun passender als bei den bisherigen Ansätzen oben, da die Residuen keinen erkennbaren Trend mehr aufweisen. Diese Komponente haben wir also erfasst und wenden uns der *Saisonalität* zu.

Allgemein erfordert das eine neue kategorische Variable, die jeden Wert der Zeitreihe einer Saison zuordnet. Die Kategorien werden dann auf Dummy-Variablen abgebildet, die als Prädiktoren in das Modell eingehen. Im hier gegebenen Fall ist das der *Monat*, den wir dem Datensatz hinzufügen. Zunächst - wir wollen eine Vorhersage nur auf Basis der Saisonalität - ist der Trend dabei als konstant angenommen.

In [ ]:
ridership_df = tsatools.add_trend(ridership_ts, trend='c')
ridership_df['Month'] = ridership_df.index.month
ridership_df

Durch die Angabe `C(Month)` nimmt *statsmodels* uns die Umwandlung in Dummies ab. Auf dieser Basis machen wir dann wie gewohnt unsere Vorhersage.

In [ ]:
train_df = ridership_df[:n_train]
valid_df = ridership_df[n_train:]
ridership_prediction_season = smf.ols(formula='Ridership ~ C(Month)', data=train_df).fit()
predicted_season = ridership_prediction_season.predict(valid_df)
print(ridership_prediction_season.summary())

Das Ergebnis stellen wir grafisch dar:

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 7.5))
ridership_prediction_season.predict(train_df).plot(ax=axes[0], color='C1')
ridership_prediction_season.predict(valid_df).plot(ax=axes[0], color='C1', linestyle='dashed')

residual = train_df.Ridership - ridership_prediction_season.predict(train_df)
residual.plot(ax=axes[1], color='C1')
residual = valid_df.Ridership - ridership_prediction_season.predict(valid_df)
residual.plot(ax=axes[1], color='C1', linestyle='dashed')
graphLayout(axes, train_df, valid_df)
plt.show()

Das Modell erfasst die Saisonalität passend, die Residuen weisen aber wieder einen Trend auf. Also fügen wir den quadratischen Trend hinzu.

In [ ]:
ridership_df = tsatools.add_trend(ridership_ts, trend='ct')
ridership_df['Month'] = ridership_df.index.month
train_df = ridership_df[:n_train]
valid_df = ridership_df[n_train:]
ridership_prediction_trend_season = smf.ols(formula='Ridership ~ trend + np.square(trend) + C(Month)', data=train_df).fit()
predicted_trend_season = ridership_prediction_trend_season.predict(valid_df)
print(ridership_prediction_trend_season.summary())

Die grafische Darstellung bietet den Eindruck eines stark verbesserten Modells.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 7.5))
ridership_prediction_trend_season.predict(train_df).plot(ax=axes[0], color='C1')
ridership_prediction_trend_season.predict(valid_df).plot(ax=axes[0], color='C1', linestyle='dashed')

residual_train = train_df.Ridership - ridership_prediction_trend_season.predict(train_df)
residual_train.plot(ax=axes[1], color='C1')
residual_valid = valid_df.Ridership - ridership_prediction_trend_season.predict(valid_df)
residual_valid.plot(ax=axes[1], color='C1', linestyle='dashed')
graphLayout(axes, train_df, valid_df)
plt.show()

Auch eine Betrachtung der Kennzahlen der Vorhersagen gegenüber dem Validierungsdatensatz im Vergleich bestätigt diesen Eindruck:

In [ ]:
print('Lineares Modell')
regressionSummary(valid_df.Ridership, predicted_linear)
print('')
print('Exponentielles Modell')
regressionSummary(valid_df.Ridership, predicted_expo)
print('')
print('Quadratisches Modell')
regressionSummary(valid_df.Ridership, predicted_poly)
print('')
print('Additiv-saisonales Modell')
regressionSummary(valid_df.Ridership, predicted_season)
print('')
print('Quadratisches Modell mit Saisonalität')
regressionSummary(valid_df.Ridership, predicted_trend_season)

## Aufgabe

Der Datensatz *a10.csv* enthält die monatlichen Absätze eines Medikamentes gegen Diabetes. Implementieren Sie ein geeignetes Modell zur Prädiktion der Absätze. Passen Sie zu diesem Zweck ggf. auch die Hilfsfunktionen zur Darstellung an.

## Autokorrelation

Zur Illustration von Autokorrelation am Beispiel der Fahrgastdaten generieren wir zunächst die verzögerten Reihen bis *lag-4* für die ersten 24 Monate.

In [ ]:
lagged_riderships_df = pd.DataFrame()
for lag in range(0, 5):
    lagged_riderships_df[f'Lag {lag}'] = train_df['1991-01-01':'1992-12-01']['Ridership'].shift(lag)

lagged_riderships_df

Die Fahrgastzahlen der ursprünglichen Zeitreihe treten in den *Lag n* jeweils *n* Monate später auf. Nun stellt sich die Frage nach der Korrelation zwischen der ursprünglichen Zeitreihe und ihren verzögerten Varianten.

In [ ]:
for lag in range(0, 5):
    print('Autokorrelation Lag {} = {}'.format(lag, train_df['1991-01-01':'1992-12-01'].Ridership.autocorr(lag=lag)))

Wir sehen schwankende Werte und einen eher niedrigen Wert für *Lag 1*. Schauen wir uns also die *Autokorrelationsfunktion* ACF der Zeitreihe an, die uns *statsmodels* für den betrachteten Zeitraum liefert und auch passend darstellt.

In [ ]:
tsaplots.plot_acf(train_df['1991-01-01':'1992-12-01'].Ridership)
plt.show()

Hier sehen wir die stärkste Autokorrelation bei Lag 6. Sie ist negativ und liegt außerhalb des 95%-Konfidenzintervalls, was auf ein verbesserungsfähiges Modell hinweist. Außerdem erkennen wir einen halbjährliche Wechsel zwischen niedrigem und hohem Fahrgastaufkommen, was wir auch schon oben in der Zeitreihe sehen konnten.

Es kann hilfreich sein, auch die Autokorrelation der Residuen zu betrachten. Eine angemessene Modellierung der Saisonalitäten sollte die Autokorrelation der Residuen unterdrücken. Stellen wir das für unser letztes (und bestes) Modell mit Trend und Saisonalität dar:

In [ ]:
tsaplots.plot_acf(residual_train['1991-01-01':'1992-12-01'])
plt.show()

Hier sehen wir Autokorrelationen und eine Saisonalität, die wir nutzen können, um Vorhersagefehler vorherzusagen. Dazu passen wir die Residuen an ein vorhandenes *ARIMA*-Modell an. Es ist hier ausreichend, einen Lag von 1 anzunehmen (`order=(1,...)`), da sich hier die unmittelbaren Nachbarn beinflussen und diese Beziehung sich auch auf die folgenden Werte auswirkt.

In [ ]:
train_residuals_arima = ARIMA(ridership_prediction_trend_season.resid, order=(1,0,0), freq='MS', trend='ct').fit()

In [ ]:
ax = ridership_prediction_trend_season.resid.plot(figsize=(9,4))
train_residuals_arima.fittedvalues.plot(ax=ax)
plt.show()

Die vorhergesagten Fehler sind nahe an den tatsächlichen Fehlern der Zeitreihe. Auch zeigt sich, dass nur noch eine vernachlässigbare Autokorrelation zwischen den Residuen der Fehler verbleibt:

In [ ]:
tsaplots.plot_acf(ridership_prediction_trend_season.resid-train_residuals_arima.fittedvalues)
plt.show()

Generieren wir eine Vorhersage des Fehlers für den April 2001 und nutzen sie zur Korrektur unserer bisherigen Vorhersage:

In [ ]:
print('Tatsächlicher Wert = {}'.format(valid_df.loc['2001-04-01'].Ridership))
print('Vorhergesagter Wert = {}'.format(predicted_trend_season.loc['2001-04-01']))
print('Fehler = {}'.format(valid_df.loc['2001-04-01'].Ridership - predicted_trend_season.loc['2001-04-01']))
print('Vorhergesagter Fehler = {}'.format(train_residuals_arima.forecast(1).loc['2001-04-01']))
print('Korrigierte Vorhersage = {}'.format(predicted_trend_season.loc['2001-04-01']+train_residuals_arima.forecast(1).loc['2001-04-01']))

Durch die spezifischen Eigenschaften der AR-Modelle eignen sie sich eher zur kurzfristigen Vorhersage der nächsten *k* Perioden. Danach greifen sie auf frühere Vorhersagen zurück.